#  Crawling through Gutenberg 
## **Written by Khaled GRIRA**
The goal here is to get the links to all of Gutenberg's books. 

First we're going to get the links to the pages then in each and everyone of them we're going to get the link to download the book and put it in a shape that's convenient to us, with the summary the name of the author the genre etc etc. 

This is for a school project: ReadingMood , PACT 3.5


In [0]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import random as rd


In [0]:
def crawlerGutenberg(original_url):
  linkscollected=[]
  source_code=requests.get(original_url)
  plain_text=source_code.text
  soup=BeautifulSoup(plain_text)
  for link in soup.findAll('ul'):
    linkscollected.append(link)
  return linkscollected  

#when using ul we're not getting the names of the authors but we can get them by looking at a book. 
# This returns the list of the books by author, but we don't have the author's name yet ! 

In [0]:
def getAuthorNameAndDate(ul):
  parent= ul.find_previous('a').find_previous('a').text
  try:
    author=re.findall("[a-zA-Z]*, [a-zA-Z]*.?,?[a-zA-Z]*",parent)[0].replace(",","")
    date=re.findall("[0-9]*-[0-9]*",parent)[0]
    return (author,date)
  except:
    try:
      author=re.findall("[a-zA-Z]*, [a-zA-Z]*.?,?[a-zA-Z]*",parent)[0].replace(",","")
      return (author,"")
    except:
      try:
        author=re.findall("[a-zA-Z]*", parent)[0]
        return (author,"")
      except:
        return ("Has a name with accents","")
 
      
  

Now we have all of the library of gutenberg. Every Author has his own string with every book he's written that's on Gutenberg. Now we're going to use regular expressions to make our data a little prettier and more convenient. Maybe we should make a dictionary. 
![This is how the section for an author is tipically written](https://i.imgur.com/boylU6w.png)

This is how the section of an author is tipically written on Gutenberg.

Most popular authors have their work translated in many langages and often there is more than one link,
we need to extract only one English one for every piece. And not take into account any author that has none in English.

![screen by Khaled](https://i.imgur.com/LWjS5VB.png)


In [0]:
debutToGutenbergLink="https://www.gutenberg.org"
def transformFalseLinkToUsableLink(link):
  return debutToGutenbergLink+link

In [0]:
def isALinkToABook(aString):
  if re.search('^/ebooks/',aString):
    return True
  else:
    return False
  
def returnAllEnglish(aWholeUl):
  author,date=getAuthorNameAndDate(aWholeUl)
  result=(author,date,[])
  for li in aWholeUl.find_all('li'):
    text=li.text
    if re.search('English',text):
      aLink=li.find('a').get('href')
      if (len(re.findall("(.*) \(English\)",text))>0):
        bookName=re.findall("(.*) \(English\)",text)[0]
        if isALinkToABook(aLink):
          realLinkToBook=transformFalseLinkToUsableLink(aLink)
          appending=(bookName,realLinkToBook)
          result[2].append(appending)
  return result
    
    

In [0]:

#print(returnAllEnglish(zeta[42]))


In [0]:
def getAllBooksRegroupedByAuthorsFromSet(set):
  result=[]
  j=0
  for Ul in set:
    result.append(returnAllEnglish(Ul))
  return result
  
  

In [0]:
basisLinkForGutenberg="https://www.gutenberg.org/browse/authors/"
letters=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]

#we're going to get the HTML code for every author with strings
allOfUls=[]
stringOfOurHtmlRaw=[]

for letter in letters:
  link=basisLinkForGutenberg+letter
  allUlForNamesStartingByLetter=crawlerGutenberg(link)
  allOfUls+=allUlForNamesStartingByLetter
  toStringOfAllUls= [str(x) for x in allUlForNamesStartingByLetter]
  stringOfOurHtmlRaw+=toStringOfAllUls
 


In [0]:
ensembleOfbooksByAuthors=getAllBooksRegroupedByAuthorsFromSet(allOfUls)

In [0]:
def hasLinkToABook(set):
  result=[]
  for x in set:
    if len(x[2])>0:
      result.append(x)
  return result

In [0]:
ensembleOfValidAuthors=hasLinkToABook(ensembleOfbooksByAuthors) # this is all of the authors that have books in english with valid links on gutenberg.
#It is an ensemble of tuples with (author, date of birth and death(when there is one), ensemble of books with titles and links) 
#now we have to create functions that can make every link we have usable for us, meaning how do I get my text from the link I do have now.



In [0]:
print(len(ensembleOfValidAuthors))
for k in range(3000,5000):
  print(ensembleOfValidAuthors[k])


In [0]:
def changeBasicLinkToTxtLink(link):
  sourceCode=requests.get(link)
  text=sourceCode.text
  soup=BeautifulSoup(text)
  try:
    linkToTxtFormat=soup.find("a",{"type":"text/plain"}).get("href")
    return ("https:"+linkToTxtFormat)
  except:
    try:
      linkToTxtFormat=soup.find("a",{"type":"text/plain; charset=utf-8"}).get("href")
      return ("https:"+linkToTxtFormat)
    except:
      return None
    

def getText(link):
  soup=BeautifulSoup(requests.get(link).text)
  return soup.find("p").text

In [0]:
def changeur(author):
  zeta= author[2]
  brut=[ (x[0],changeBasicLinkToTxtLink(x[1]) ) for x in zeta]
  result=[author[0],author[1],brut]
  return result

In [0]:
for k in range(100,200):
  brut=ensembleOfValidAuthors[k]
  print(brut)
  print(brut[2])
  for x in brut[2]:
    print(x[0])
    print(x[1])
    print(changeBasicLinkToTxtLink(x[1]))


In [0]:
print(FinalLibrary)

In [0]:
import sqlite3
newDb=sqlite3.connect('TableBooks.sql')
cursor=newDb.cursor()

In [0]:
def unvectorOfAuthorToJustBooks(author):
  result=[]
  for x in author[2]:
    result.append((author[0],author[1],x[0],x[1],0))
  return result

In [19]:
cursor.execute('CREATE TABLE TableOfBooks (Author text, Date text, Title text, Link text, Treated int)')

In [18]:
cursor.execute('DROP TABLE TableOfBooks')

In [0]:
def insertIntoDatabase(modifiedBook,curseur):
  commandString="INSERT INTO TableOfBooks VALUES (?,?,?,?,?)"
  curseur.execute(commandString,modifiedBook)
  
  

In [0]:
for author in ensembleOfValidAuthors:
  y=unvectorOfAuthorToJustBooks(author)
  for book in y:
    insertIntoDatabase(book,cursor)
 
 

In [0]:
from google.colab import files
field_names=[val[0] for val in cursor.description]

with open('example.txt', 'w') as f:
  for row in cursor.execute("SELECT * FROM TableOfBooks"):
    stringAdd=""
    for k in range(5):
      stringAdd+=field_names[k]+"= "+str(row[k])+", "
    stringAdd+="\n"
    f.write(stringAdd)
    

files.download('example.txt')